## Gmarket 베스트 상품 200개 크롤링
- robots.txt 결과 모두 허용...
- 상품명, 원가, 판매가
- 썸네일 이미지

In [1]:
import requests

In [2]:
response = requests.get('http://corners.gmarket.co.kr/Bestsellers')
response

<Response [200]>

In [4]:
doc = BeautifulSoup(response.text, 'html5lib')

In [5]:
selector = '#gBestWrap > div > div:nth-child(5) > div:nth-child(3) > ul > li'
elements = doc.select(selector)
len(elements)

200

- 상품코드, 제목, 판매가

In [50]:
int(re.search('goodscode=(\d+)', elements[0].select_one('a').get('href')).groups()[0])

1992687537

In [36]:
import re
elements[0].select_one('.itemname').text

'[브리타]막스트라 플러스 필터 3개월분'

In [40]:
elements[0].select_one('.s-price').text.strip().split('\n')[0]

'20,900원'

In [52]:
datas = []
for element in elements:
    datas.append({
        'code' :int(re.search('goodscode=(\d+)', element.select_one('a').get('href')).groups()[0]),
        'title':element.select_one('.itemname').text,
        'price':element.select_one('.s-price').text.strip().split('\n')[0],
    })
items_df = pd.DataFrame(datas)
items_df.tail()

,code,title,price
195,1986796870,꿀당도 서귀포 한라봉 4.5kg내외/(25-28과/중소과),"16,900원"
196,1552234801,[스팸]스팸 클래식 200g 10개,"22,900원"
197,805459425,[슈프림]남녀공용 후드티 남성여성 빅사이즈티셔츠 기모맨투맨,"9,900원"
198,1980690983,[삼양]삼양 육개장 1박스 (24개입) /컵라면/라면,"15,900원"
199,1688878141,[팬콧키즈]22%쿠폰 패딩 플리스 기모 티셔츠 팬츠 상하복 바지,"17,900원"


### 이미지 크롤링

In [56]:
for idx, row in items_df.iterrows():
    link = 'http://gdimg.gmarket.co.kr/{}/still/300'.format(row['code'])
    title = row['title'] + '.png'
    title = title.replace('/', '')
    print(link, title)

http://gdimg.gmarket.co.kr/1992687537/still/300 [브리타]막스트라 플러스 필터 3개월분.png
http://gdimg.gmarket.co.kr/1564736958/still/300 NEW 리뉴얼 그로미미 PPSU스테인리스 빨대컵 1+1.png
http://gdimg.gmarket.co.kr/2009842804/still/300 마카롱리빙박스72L4개 정리함 수납박스 장난감정리함.png
http://gdimg.gmarket.co.kr/1953059281/still/300 에티카  KF94 중형대형 30매 블랙화이트.png
http://gdimg.gmarket.co.kr/1884627218/still/300 [실리쿡][실리쿡] 하프트레이 12개 세트(특대4 + 대4개 + 소4개).png
http://gdimg.gmarket.co.kr/1602730714/still/300 정직한 젓갈   비빔오징어젓 1kg 밥도둑 쿠폰가 9620원.png
http://gdimg.gmarket.co.kr/1798397615/still/300 [케피](125일단하루1+1+1)케피버블 클렌저 목욕놀이거품.png
http://gdimg.gmarket.co.kr/1999349775/still/300 해남 꿀 고구마 3kg 대(300g이상).png
http://gdimg.gmarket.co.kr/1729698859/still/300 [3M]표준형 베이직 물걸레 더블액션 청소포 180매.png
http://gdimg.gmarket.co.kr/2011066048/still/300 숨마 주니어 중학 국어 비문학독해 123어휘력123문법연습 연습12.png
http://gdimg.gmarket.co.kr/1910040585/still/300 [프렌치카페]프렌치카페 커피믹스 300T +10T.png
http://gdimg.gmarket.co.kr/1962034682/still/300 [오늘]현대제지 오늘 천연펄프 키친타올 125매4롤4팩 총16롤.png
http:

In [57]:
!mkdir datas

In [58]:
def download(title, link):
    # stream : 데이터를 잘게 쪼개서
    # 대용량 데이터일 경우, 전송되는 데이터 중 한 번 깨지면 처음부터 다시 보냄
    response = requests.get(link, stream=True) 
    path = "./datas/{}".format(title)
    
    with open(path, 'wb') as f:
        # 파일 분할해서 저장하는 것
        for chunk in response.iter_content(chunk_size=1024):
            if chunk:
                f.write(chunk)

In [59]:
for idx, row in items_df.iterrows():
    link = 'http://gdimg.gmarket.co.kr/{}/still/300'.format(row['code'])
    title = row['title'] + '.png'
    title = title.replace('/', '')
    print(title, link)
    download(title, link)

[브리타]막스트라 플러스 필터 3개월분.png http://gdimg.gmarket.co.kr/1992687537/still/300
NEW 리뉴얼 그로미미 PPSU스테인리스 빨대컵 1+1.png http://gdimg.gmarket.co.kr/1564736958/still/300
마카롱리빙박스72L4개 정리함 수납박스 장난감정리함.png http://gdimg.gmarket.co.kr/2009842804/still/300
에티카  KF94 중형대형 30매 블랙화이트.png http://gdimg.gmarket.co.kr/1953059281/still/300
[실리쿡][실리쿡] 하프트레이 12개 세트(특대4 + 대4개 + 소4개).png http://gdimg.gmarket.co.kr/1884627218/still/300
정직한 젓갈   비빔오징어젓 1kg 밥도둑 쿠폰가 9620원.png http://gdimg.gmarket.co.kr/1602730714/still/300
[케피](125일단하루1+1+1)케피버블 클렌저 목욕놀이거품.png http://gdimg.gmarket.co.kr/1798397615/still/300
해남 꿀 고구마 3kg 대(300g이상).png http://gdimg.gmarket.co.kr/1999349775/still/300
[3M]표준형 베이직 물걸레 더블액션 청소포 180매.png http://gdimg.gmarket.co.kr/1729698859/still/300
숨마 주니어 중학 국어 비문학독해 123어휘력123문법연습 연습12.png http://gdimg.gmarket.co.kr/2011066048/still/300
[프렌치카페]프렌치카페 커피믹스 300T +10T.png http://gdimg.gmarket.co.kr/1910040585/still/300
[오늘]현대제지 오늘 천연펄프 키친타올 125매4롤4팩 총16롤.png http://gdimg.gmarket.co.kr/1962034682/still/300
[참도깨비

브레스마스크 KF-94 대형  100매입.png http://gdimg.gmarket.co.kr/1955584051/still/300
아이 스스로 성장하는 똑똑하게 레벨 업 세트(전7권).png http://gdimg.gmarket.co.kr/2005702245/still/300
[크리넥스]크리넥스 KF80 입체형 마스크 소형 10PX4개.png http://gdimg.gmarket.co.kr/2013024635/still/300
[폴햄]폴햄 맨투맨후드 외 이너 모음.png http://gdimg.gmarket.co.kr/1904145773/still/300
[릴리유]릴리유 퓨어그린 아기물티슈 캡 80매 10팩 E.png http://gdimg.gmarket.co.kr/1658859485/still/300
3중 필터 샤워기 (+리필필터 10P).png http://gdimg.gmarket.co.kr/2005874455/still/300
엄마애바다 국내가공 백진미채 1kg.png http://gdimg.gmarket.co.kr/1636284317/still/300
로지텍 G304 무선 게이밍 마우스 병행 밀봉박스새상품.png http://gdimg.gmarket.co.kr/1468424485/still/300
얼라이브즈 슬라이딩 리빙박스 3P SET.png http://gdimg.gmarket.co.kr/1872366168/still/300
[한샘]양날개 튼튼 빨래 건조대 + 사은품 3종택1.png http://gdimg.gmarket.co.kr/2006318469/still/300
[휠라]휠라볼란테 98 스웨이드 FS1HTB1167XWRD.png http://gdimg.gmarket.co.kr/2004307722/still/300
[뉴발란스]본사최대 75% 혜택 신학기가방맨투맨운동화기획전.png http://gdimg.gmarket.co.kr/2004425139/still/300
(행사상품)믿고먹는_안심농협쌀_20KG 포 .png http://gdimg.gma

OSError: [Errno 22] Invalid argument: './datas/|서울 중구| 서울신라호텔(신라신라호텔서울신라호텔수영장호텔추천특1급중구호텔호텔신라).png'